In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
from ipywidgets import interact, interactive

In [2]:
# output directory
direction = "up"
species = "MR"
now = datetime.datetime.now()
line_enrichment = f'../data/line_enrichment_{direction}_{species}_{now.strftime("%y%m")}'
os.makedirs(line_enrichment, exist_ok=True)

In [3]:
def count_goslim(df, file_name: str):
    unique_gene_pairs = df.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_mouse', 'Gene stable ID_rice'])
    goslim_counts = unique_gene_pairs.groupby(['GOSlim GOA Accession(s)', 'GOSlim GOA Description', 'GOSlim_domain']).size().reset_index(name='counts')
    goslim_counts = goslim_counts.sort_values(by='counts', ascending=False).copy()
    goslim_counts.reset_index(drop=True, inplace=True)
    results_directory = f'../data/{line_enrichment}/goslim_correspondence_counts_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    goslim_counts.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    return goslim_counts

In [4]:
merged_df_goslim_pd = pd.read_csv('../data/GOSlim_merge_MR/GOslim_merge_common_goslim_correspondence_all.tsv', 
                                  sep='\t',
                                  low_memory=False)
merged_df_up = pd.read_csv(f'../data/circos_{direction}_{species}_2312/combined_goslim_up.tsv', sep='\t')

display(merged_df_goslim_pd, merged_df_up)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,Gene stable ID_mouse,Gene stable ID_rice
0,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0100700
1,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0101150
2,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0101300
3,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0101600
4,GO:0003723,RNA binding,molecular_function,ENSMUSG00000064336,Os01g0118100
...,...,...,...,...,...
183104384,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0514500
183104385,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0530100
183104386,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0546800
183104387,GO:0030312,external encapsulating structure,cellular_component,ENSMUSG00000026938,Os12g0563700


,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,Gene stable ID_rice,Gene stable ID_mouse,Chromosome/scaffold name_rice,start1 (bp),end1 (bp),Chromosome/scaffold name_mouse,start2 (bp),end2 (bp),HN-score(HN5)_rice,HN-score(HN5)_mouse
0,GO:0005634,nucleus,cellular_component,Os03g0268600,ENSMUSG00000042745,R_3,8921424,8923437,M_2,152578171,152579330,59,14
1,GO:0005634,nucleus,cellular_component,Os03g0701100,ENSMUSG00000020427,R_3,28138905,28141758,M_11,7156086,7163923,103,16
2,GO:0005634,nucleus,cellular_component,Os03g0131800,ENSMUSG00000020427,R_3,1815345,1816503,M_11,7156086,7163923,65,16
3,GO:0005634,nucleus,cellular_component,Os08g0546800,ENSMUSG00000020427,R_8,27382872,27384470,M_11,7156086,7163923,73,16
4,GO:0005634,nucleus,cellular_component,Os03g0710500,ENSMUSG00000020427,R_3,28647086,28649570,M_11,7156086,7163923,74,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16724,GO:0005730,nucleolus,cellular_component,Os06g0592500,ENSMUSG00000005483,R_6,23293383,23294073,M_8,84334822,84339282,118,71
16725,GO:0005730,nucleolus,cellular_component,Os06g0592500,ENSMUSG00000026628,R_6,23293383,23294073,M_1,190902493,190950236,118,78
16726,GO:0005777,peroxisome,cellular_component,Os06g0253100,ENSMUSG00000070985,R_6,7940956,7941680,M_4,49447105,49473912,129,14
16727,GO:0005777,peroxisome,cellular_component,Os07g0529000,ENSMUSG00000070985,R_7,20691213,20693521,M_4,49447105,49473912,59,14


In [5]:
gosilm_correspondence_up_counts = count_goslim(merged_df_up, f'goslim_correspondence_counts_{direction}.tsv')
goslim_correspondence_all_counts = count_goslim(merged_df_goslim_pd, 'goslim_correspondence_counts_all.tsv')

display(gosilm_correspondence_up_counts, goslim_correspondence_all_counts)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts
0,GO:0005634,nucleus,cellular_component,4150
1,GO:0003824,catalytic activity,molecular_function,4136
2,GO:0005829,cytosol,cellular_component,1326
3,GO:0048856,anatomical structure development,biological_process,1222
4,GO:0005886,plasma membrane,cellular_component,858
5,GO:0016787,hydrolase activity,molecular_function,798
6,GO:0005576,extracellular region,cellular_component,610
7,GO:0003677,DNA binding,molecular_function,567
8,GO:0005783,endoplasmic reticulum,cellular_component,504
9,GO:0036211,protein modification process,biological_process,462


,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts
0,GO:0003824,catalytic activity,molecular_function,56414781
1,GO:0005634,nucleus,cellular_component,46028115
2,GO:0005886,plasma membrane,cellular_component,13465191
3,GO:0036211,protein modification process,biological_process,10391837
4,GO:0016740,transferase activity,molecular_function,9615454
5,GO:0016787,hydrolase activity,molecular_function,7777545
6,GO:0048856,anatomical structure development,biological_process,7211128
7,GO:0005829,cytosol,cellular_component,5489678
8,GO:0003677,DNA binding,molecular_function,4990802
9,GO:0003723,RNA binding,molecular_function,3071712


In [6]:
# for debugging purpose
total_counts_up = gosilm_correspondence_up_counts['counts'].sum()
total_counts_all = goslim_correspondence_all_counts['counts'].sum()

unique_rows_up = merged_df_up.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_mouse', 'Gene stable ID_rice']).shape[0]
unique_rows_all = merged_df_goslim_pd.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_mouse', 'Gene stable ID_rice']).shape[0]

print(f"Total counts: {unique_rows_all}")
print(f"up counts: {unique_rows_up}")
print(f"Counts match rows: {unique_rows_up == total_counts_up}")

Total counts: 183104389
up counts: 16729
Counts match rows: True


In [7]:
# fold enrichment
def fold_enrichment(df_goslim_up_counts, df_goslim_all_counts, unique_genes_up, unique_genes_all, file_name: str):
    """_summary_
    Args:
        df_goslim_up_counts (dataframe): _description_
        df_goslim_all_counts (dataframe): _description_
        unique_genes_up (): _description_
        unique_genes_all (dataframe): _description_
        file_name (str): _description_

    Returns:
        dataframe : _description_
    """
    merged_df = pd.merge(df_goslim_up_counts, 
                         df_goslim_all_counts,
                         on=['GOSlim GOA Accession(s)', 'GOSlim GOA Description', 'GOSlim_domain'], 
                         how='right', # if there is no match, fill with NaN or 0
                         suffixes=('_up', '_all'))
    
    merged_df['counts_up'] = merged_df['counts_up'].fillna(0)
    merged_df['counts_all'] = merged_df['counts_all'].fillna(0)
    merged_df['up_ratio'] = merged_df['counts_up'] / unique_genes_up
    merged_df['all_ratio'] = merged_df['counts_all'] / unique_genes_all
    merged_df['fold_enrichment'] = merged_df['up_ratio'] / merged_df['all_ratio']
    merged_df.replace([np.inf, -np.inf], np.nan, inplace=True) # inf -> nan
    results_directory = f'../data/{line_enrichment}/goslim_fold_enrichment_correspondence_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    merged_df.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    
    return merged_df


goslim_correspondence_enrichment = fold_enrichment(gosilm_correspondence_up_counts, 
                                                   goslim_correspondence_all_counts, 
                                                   unique_rows_up, 
                                                   unique_rows_all, 
                                                   f'goslim_correspondence_fold_enrichment_{direction}.tsv')
display(goslim_correspondence_enrichment)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts_up,counts_all,up_ratio,all_ratio,fold_enrichment
0,GO:0003824,catalytic activity,molecular_function,4136.0,56414781,0.247235,0.308102,0.802447
1,GO:0005634,nucleus,cellular_component,4150.0,46028115,0.248072,0.251376,0.986856
2,GO:0005886,plasma membrane,cellular_component,858.0,13465191,0.051288,0.073538,0.697435
3,GO:0036211,protein modification process,biological_process,462.0,10391837,0.027617,0.056754,0.486607
4,GO:0016740,transferase activity,molecular_function,306.0,9615454,0.018292,0.052514,0.348322
5,GO:0016787,hydrolase activity,molecular_function,798.0,7777545,0.047702,0.042476,1.123024
6,GO:0048856,anatomical structure development,biological_process,1222.0,7211128,0.073047,0.039383,1.854799
7,GO:0005829,cytosol,cellular_component,1326.0,5489678,0.079264,0.029981,2.643781
8,GO:0003677,DNA binding,molecular_function,567.0,4990802,0.033893,0.027257,1.243488
9,GO:0003723,RNA binding,molecular_function,50.0,3071712,0.002989,0.016776,0.178163


## Q-value

In [8]:
def calculate_p_q_values(df_enrichment, unique_correspondence_up, unique_correspondence_all, file_name: str):
    p_values = []
    df_enrichment['counts_up'] = df_enrichment['counts_up'].astype(int) # float(0.0) to int(0)
    for index, row in df_enrichment.iterrows():
        if row['counts_up'] == 0:
            p_values.append(1.0)
        else:
            observed_up_correspondence = row['counts_up'] # create contingency table
            observed_all_correspondence = row['counts_all']
            observed_not_up_correspondence = observed_all_correspondence - observed_up_correspondence
            total_not_up_correspondence = unique_correspondence_all - unique_correspondence_up
            table = [
            [observed_up_correspondence, unique_correspondence_up - observed_up_correspondence],
            [observed_not_up_correspondence, total_not_up_correspondence - observed_not_up_correspondence]
            ]
            _, p_value = fisher_exact(table, alternative='greater') # Fisher's exact test
            p_values.append(p_value)
    
    _, q_values, _, _ = multipletests(p_values, 
                                      alpha=0.05, 
                                      method='fdr_bh') # Benjamini/Hochberg method
    
    df_enrichment['p_value'] = p_values
    df_enrichment['q_value'] = q_values

    results_directory = f'../data/{line_enrichment}/goslim_correspondence_q_values_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    df_enrichment['GOSlim'] = df_enrichment['GOSlim GOA Accession(s)'] + ": " + df_enrichment['GOSlim GOA Description']
    cols = df_enrichment.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df_enrichment = df_enrichment[cols]
    df_enrichment.drop(['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], axis=1)
    df_enrichment = df_enrichment[df_enrichment['counts_up'] != 0]
    df_enrichment = df_enrichment.sort_values(by='fold_enrichment', ascending=False)
    df_enrichment.reset_index(drop=True, inplace=True)
    df_enrichment.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)

    return df_enrichment

goslim_correspondence_with_p_q = calculate_p_q_values(goslim_correspondence_enrichment.copy(),
                                                      unique_rows_up,
                                                      unique_rows_all, 
                                                      f'goslim_correspondence_fold_enrichment_p_q_{direction}.tsv')


display(goslim_correspondence_with_p_q)

,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts_up,counts_all,up_ratio,all_ratio,fold_enrichment,p_value,q_value
0,GO:0008289: lipid binding,GO:0008289,lipid binding,molecular_function,147,257535,0.008787,0.001406,6.247552,1.084662e-65,7.375700e-65
1,GO:0006091: generation of precursor metabolite...,GO:0006091,generation of precursor metabolites and energy,biological_process,77,243045,0.004603,0.001327,3.467630,9.441559e-20,4.585900e-19
2,GO:0005783: endoplasmic reticulum,GO:0005783,endoplasmic reticulum,cellular_component,504,1609508,0.030127,0.008790,3.427411,1.265856e-118,1.434636e-117
3,GO:0005576: extracellular region,GO:0005576,extracellular region,cellular_component,610,2069023,0.036464,0.011300,3.226958,2.927792e-132,4.977246e-131
4,GO:0005829: cytosol,GO:0005829,cytosol,cellular_component,1326,5489678,0.079264,0.029981,2.643781,1.413623e-213,4.806317e-212
5,GO:0005773: vacuole,GO:0005773,vacuole,cellular_component,64,270510,0.003826,0.001477,2.589557,3.137256e-11,1.066667e-10
6,GO:0140110: transcription regulator activity,GO:0140110,transcription regulator activity,molecular_function,396,1948470,0.023671,0.010641,2.224489,1.240937e-45,7.031977e-45
7,GO:0005975: carbohydrate metabolic process,GO:0005975,carbohydrate metabolic process,biological_process,120,655182,0.007173,0.003578,2.004694,4.773265e-12,1.803234e-11
8,GO:0048856: anatomical structure development,GO:0048856,anatomical structure development,biological_process,1222,7211128,0.073047,0.039383,1.854799,5.857880e-90,4.979198e-89
9,GO:0005739: mitochondrion,GO:0005739,mitochondrion,cellular_component,322,2075124,0.019248,0.011333,1.698402,8.449761e-19,3.591148e-18


In [9]:
#data preprocessing for plotting
goslim_correspondence_with_p_q.rename(columns={'counts_up': 'line counts'}, inplace=True)

if 'p_value' in goslim_correspondence_with_p_q.columns:
    goslim_correspondence_with_p_q['-log10(p-value)'] = -np.log10(goslim_correspondence_with_p_q['p_value'])
    
if 'q_value' in goslim_correspondence_with_p_q.columns:
    goslim_correspondence_with_p_q['-log10(q-value)'] = -np.log10(goslim_correspondence_with_p_q['q_value'])


cc = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'cellular_component']
bp = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'biological_process']
mf = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'molecular_function']

cc.reset_index(drop=True, inplace=True)
bp.reset_index(drop=True, inplace=True)
mf.reset_index(drop=True, inplace=True)

display(cc, bp, mf)

,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,up_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0005783: endoplasmic reticulum,GO:0005783,endoplasmic reticulum,cellular_component,504,1609508,0.030127,0.008790,3.427411,1.265856e-118,1.434636e-117,1.178976e+02,116.843258
1,GO:0005576: extracellular region,GO:0005576,extracellular region,cellular_component,610,2069023,0.036464,0.011300,3.226958,2.927792e-132,4.977246e-131,1.315335e+02,130.303011
2,GO:0005829: cytosol,GO:0005829,cytosol,cellular_component,1326,5489678,0.079264,0.029981,2.643781,1.413623e-213,4.806317e-212,2.128497e+02,211.318188
3,GO:0005773: vacuole,GO:0005773,vacuole,cellular_component,64,270510,0.003826,0.001477,2.589557,3.137256e-11,1.066667e-10,1.050345e+01,9.971971
4,GO:0005739: mitochondrion,GO:0005739,mitochondrion,cellular_component,322,2075124,0.019248,0.011333,1.698402,8.449761e-19,3.591148e-18,1.807316e+01,17.444767
5,GO:0005777: peroxisome,GO:0005777,peroxisome,cellular_component,2,17818,0.000120,0.000097,1.228570,4.840561e-01,1.000000e+00,3.151043e-01,-0.000000
6,GO:0005634: nucleus,GO:0005634,nucleus,cellular_component,4150,46028115,0.248072,0.251376,0.986856,8.399103e-01,1.000000e+00,7.576711e-02,-0.000000
7,GO:0005654: nucleoplasm,GO:0005654,nucleoplasm,cellular_component,82,1255426,0.004902,0.006856,0.714910,9.994528e-01,1.000000e+00,2.377294e-04,-0.000000
8,GO:0005886: plasma membrane,GO:0005886,plasma membrane,cellular_component,858,13465191,0.051288,0.073538,0.697435,1.000000e+00,1.000000e+00,-0.000000e+00,-0.000000
9,GO:0005856: cytoskeleton,GO:0005856,cytoskeleton,cellular_component,35,661920,0.002092,0.003615,0.578750,9.998519e-01,1.000000e+00,6.431414e-05,-0.000000


,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,up_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0006091: generation of precursor metabolite...,GO:0006091,generation of precursor metabolites and energy,biological_process,77,243045,0.004603,0.001327,3.467630,9.441559e-20,4.585900e-19,19.024956,18.338575
1,GO:0005975: carbohydrate metabolic process,GO:0005975,carbohydrate metabolic process,biological_process,120,655182,0.007173,0.003578,2.004694,4.773265e-12,1.803234e-11,11.321184,10.743948
2,GO:0048856: anatomical structure development,GO:0048856,anatomical structure development,biological_process,1222,7211128,0.073047,0.039383,1.854799,5.857880e-90,4.979198e-89,89.232260,88.302841
3,GO:0006629: lipid metabolic process,GO:0006629,lipid metabolic process,biological_process,162,1429560,0.009684,0.007807,1.240342,4.259509e-03,1.114025e-02,2.370640,1.953105
4,GO:0030154: cell differentiation,GO:0030154,cell differentiation,biological_process,76,1302168,0.004543,0.007112,0.638815,9.999903e-01,1.000000e+00,0.000004,-0.000000
5,GO:0036211: protein modification process,GO:0036211,protein modification process,biological_process,462,10391837,0.027617,0.056754,0.486607,1.000000e+00,1.000000e+00,-0.000000,-0.000000


,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,up_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0008289: lipid binding,GO:0008289,lipid binding,molecular_function,147,257535,0.008787,0.001406,6.247552,1.084662e-65,7.375700e-65,64.964706,64.132197
1,GO:0140110: transcription regulator activity,GO:0140110,transcription regulator activity,molecular_function,396,1948470,0.023671,0.010641,2.224489,1.240937e-45,7.031977e-45,44.906250,44.152923
2,GO:0003677: DNA binding,GO:0003677,DNA binding,molecular_function,567,4990802,0.033893,0.027257,1.243488,2.012027e-07,6.218991e-07,6.696366,6.206280
3,GO:0016787: hydrolase activity,GO:0016787,hydrolase activity,molecular_function,798,7777545,0.047702,0.042476,1.123024,5.258864e-04,1.490011e-03,3.279108,2.826810
4,GO:0005215: transporter activity,GO:0005215,transporter activity,molecular_function,150,1687420,0.008966,0.009216,0.972964,6.429360e-01,1.000000e+00,0.191832,-0.000000
5,GO:0005198: structural molecule activity,GO:0005198,structural molecule activity,molecular_function,40,488130,0.002391,0.002666,0.896919,7.746165e-01,1.000000e+00,0.110913,-0.000000
6,GO:0003824: catalytic activity,GO:0003824,catalytic activity,molecular_function,4136,56414781,0.247235,0.308102,0.802447,1.000000e+00,1.000000e+00,-0.000000,-0.000000
7,GO:0045182: translation regulator activity,GO:0045182,translation regulator activity,molecular_function,1,26832,0.000060,0.000147,0.407921,9.138572e-01,1.000000e+00,0.039122,-0.000000
8,GO:0016740: transferase activity,GO:0016740,transferase activity,molecular_function,306,9615454,0.018292,0.052514,0.348322,1.000000e+00,1.000000e+00,-0.000000,-0.000000
9,GO:0003723: RNA binding,GO:0003723,RNA binding,molecular_function,50,3071712,0.002989,0.016776,0.178163,1.000000e+00,1.000000e+00,-0.000000,-0.000000


In [10]:
# enrichment result plot function
current_fig = None # global variable

def plot_goslim_dotplot_with_lines(data, col1, goslim_col, col2, col3, ylabel_suffix='', width=6, height=9, palette="flare"):

    # 1. figure size
    global current_fig
    figsize = (width, height)
    current_fig, ax = plt.subplots(figsize=figsize)

    # 2. color normalization
    norm = mcolors.Normalize(vmin=data[col3].min(), vmax=data[col3].max())

    # 3.color palette
    color_palette = sns.color_palette(palette, as_cmap=True)

    # 4.plotting
    sns.scatterplot(
        data=data,
        x=col1,
        y=goslim_col,
        size=col2, 
        hue=col3,     
        palette=palette, 
        legend='brief',
        ax=ax
    )
    # 5.lines 
    for index, row in data.iterrows():
        ax.plot([0, row[col1]], [row[goslim_col], row[goslim_col]], color=color_palette(norm(row[col3])), lw=1)
    
    ax.grid(color='b', linestyle=':', linewidth=0.1)
    ax.set_xlabel(col1)
    ylabel_text = 'Common GOSlim Terms' + ('' + ylabel_suffix  if ylabel_suffix else '' )
    ax.set_ylabel(ylabel_text)

    plt.show()

# execute only this function to plot
# plot_goslim_dotplot_with_lines(goslim_correspondence_with_p_q.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix='') # add copy() to avoid SettingWithCopyWarning
# plot_goslim_dotplot_with_lines(cc.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Cellular Component)')
# plot_goslim_dotplot_with_lines(bp.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Biological Process)')
# plot_goslim_dotplot_with_lines(mf.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Molecular Function)')

In [11]:
# interactive setting
filename_text = widgets.Text(value='', description='File Name:')
png_directory = f'../data/{line_enrichment}/png'
os.makedirs(png_directory, exist_ok=True)

def save_plot(button):
    global current_fig
    if current_fig:
        filename = filename_text.value
        if filename:  
            current_fig.savefig(f'{png_directory}/{filename}.png', dpi=800, bbox_inches='tight')
        else:
            current_fig.savefig(f'{png_directory}/enrichment_result.png', dpi=800, bbox_inches='tight')


save_button = widgets.Button(description="Save Plot")
save_button.on_click(save_plot)

def interactive_plot(data):
    w = interactive(
        plot_goslim_dotplot_with_lines,
        data=widgets.fixed(data),
        col1=widgets.Dropdown(options= [('fold enrichment', 'fold_enrichment'), 
                                        ('-log10(p-value)', '-log10(p-value)'), 
                                        ('-log10(q-value)', '-log10(q-value)')], 
                                        value='fold_enrichment', 
                                        description='x-axis:'),
        goslim_col=widgets.fixed('GOSlim'),
        col2=widgets.Dropdown(options=data.columns.tolist(), value='line counts', description='size:'),
        col3=widgets.Dropdown(options= [('fold enrichment', 'fold_enrichment'), 
                                        ('-log10(p-value)', '-log10(p-value)'), 
                                        ('-log10(q-value)', '-log10(q-value)')], 
                                        value='-log10(q-value)', 
                                        description='hue:'),
        ylabel_suffix=widgets.Dropdown(
            options=[('', ''), 
                     ('cc', '(Cellular Component)'), 
                     ('bp', '(Biological Process)'), 
                     ('mf', '(Molecular Function)')],
            value='',
            description='y-label suffix:'),
        width=widgets.IntSlider(min=1, max=30, step=1, value=6, description='width:', readout_format='0.0f'),
        height=widgets.IntSlider(min=1, max=30, step=1, value=9, description='height:', readout_format='0.0f'),
        palette=widgets.Dropdown(options=['flare', 'magma', 'viridis'], value='flare', description='color palette:')
        
    )
    display(w, filename_text, save_button)

In [12]:
interactive_plot(goslim_correspondence_with_p_q.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [13]:
interactive_plot(cc.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_up_all', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [14]:
interactive_plot(bp.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_up_cc', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [15]:
interactive_plot(mf.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_up_bp', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())